In [1]:
#최종본 ) 추가요소- 전체 글의 개수 가져와서 5페이지씩 몇개인지 보고 몇번 돌지 i변수에 넣어주기.
import json
from bs4 import BeautifulSoup
import html
import urllib.request
import requests
import time
from selenium import webdriver
import sqlite3
import datetime

In [2]:
con = sqlite3.connect("TheMinJoo.db")
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        url TEXT NOT NULL);
        
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table1_id INTEGER NOT NULL,
        content TEXT NOT NULL); 
    """)
con.commit()

In [7]:
# with open("account.json") as fp:
#     account = json.load(fp)
    

In [5]:
driver = webdriver.Chrome()
driver.get("http://theminjoo.kr/briefingList.do")

In [6]:
i = 0
while(i<3):  #5개page를 묶음으로 몇번돌지 정하기.
    
    for page in range(2,7):
        if(page == 6):
            page=1
            #5개단위로 넘기기.
            driver.find_element_by_xpath('//*[@id="content"]/div[3]/a[3]').click()
            i+=1
            
        li = driver.find_element_by_css_selector('.paging ul li a')   

        #각 안에 들어간 10개의 링크들
        for _ in driver.find_elements_by_css_selector(".bbs_list .tal > a"):
            if len(_.get_attribute("href"))>1:
                cur.execute("INSERT INTO table1(title,url) VALUES(?,?)",
                           [_.text,_.get_attribute("href"),])
                con.commit()
        cur.execute("SELECT * FROM table1")

        #1,2,3페이지. 
        driver.find_element_by_xpath('//*[@id="content"]/div[3]/ul[%s]/li/a' %page).click()
        

driver.execute_script("javascript:window.open('about:blank')")
driver.switch_to_window(driver.window_handles[-1])

for _ in  cur.fetchall():
    driver.get(_[2])
    cur.execute("INSERT INTO table2(table1_id, content)VALUES(?,?)",
               [_[0],driver.find_element_by_css_selector('div.cont').text])
    con.commit()
    
driver.close()

        

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: use driver.switch_to.window instead


150개에 220초 약 1개당 1.5초